In [1]:
'''
data_preprocess.py
this code is for making dataframe using mgf file. 
dataframe will have columns of sequence, mass, spectrum begin idx, end idx ...

and the rest are test code
'''


import numpy as np
import pandas as pd

f = open("/home/workplace/ms2/embedding/LIBRARY_AUGMENT-adfc8252-download_filtered_mgf_library-main.mgf")
content = f.readlines()
f.close()

In [2]:
import pickle

len(content)

259456768

In [4]:
with open("all.pkl", "wb") as f:
    pickle.dump(content, f)

: 

In [ ]:
with open("all.pkl", "rb") as f1:
    list1 = pickle.load(f1)

In [ ]:
np.__version__

In [ ]:
len(content)

In [ ]:

for i,a in enumerate(content):
    if i < 5568:
        print(a)
    else:
        break

In [ ]:
begin_index = []
end_idx = []
sequence = []
charge = []
pepmass = []
scan = []
score = []
fdr = []
protein = []
for i,line in enumerate(content):
    if line == "BEGIN IONS\n":
        begin_index.append(i)
    elif line == "END IONS\n":
        end_idx.append(i)
    elif "PEPMASS" in line:
        pepmass.append(line[8:-1])
    elif "CHARGE" in line:
        charge.append(line[7:-1])
    elif "SCANS" in line:
        scan.append(line[6:-1])
    elif "SEQ" in line:
        sequence.append(line[4:-1])
    elif "SCORE" in line:
        score.append(line[6:-1])
    elif "FDR" in line:
        fdr.append(line[4:-1])
    elif "PROTEIN" in line:
        protein.append(line[8:-1])

                


In [ ]:
df = pd.DataFrame({"begin_index" : begin_index,
                   "end_idx" : end_idx,
                   "pepmass" : pepmass,
                   "charge" : charge,
                   "scan" : scan,
                   "sequence" : sequence,
                   "score" : score,
                   "fdr" : fdr,
                   "protein" : protein})
df.shape


In [ ]:
df.to_csv("data_processed.tsv", sep="\t",index=False)

In [2]:
df = pd.read_csv("data_processed.tsv", delimiter="\t")

In [3]:
df.shape

(2140865, 9)

In [ ]:
content[df.loc[0, "begin_index"]+16:df.loc[0, "end_idx"]]

In [ ]:
import numpy as np
import numba as nb

In [ ]:
def get_spectrum(mgf, begin, end):
    spectrum = mgf[begin + 16: end]
    mz = [float(i.split("\t")[0].strip()) for i in spectrum]
    intensity = [float(i.split("\t")[1].strip()) for i in spectrum]
    mz = np.array(mz)
    intensity = np.array(intensity)
    return mz, intensity

@nb.njit
def _norm_intensity(spectrum_intensity: np.ndarray) -> np.ndarray:
    """
    Normalize spectrum peak intensities.

    Parameters
    ----------
    spectrum_intensity : np.ndarray
        The spectrum peak intensities to be normalized.

    Returns
    -------
    np.ndarray
        The normalized peak intensities.
    """
    return spectrum_intensity / np.linalg.norm(spectrum_intensity)


print(get_spectrum(content, 0,42))

In [ ]:
mz, intensity = get_spectrum(content, 0, 42)
print(intensity)

intensity_norm = _norm_intensity(intensity)
print(intensity_norm)


In [ ]:
from spectrum_utils.spectrum import MsmsSpectrum

In [ ]:
spectrum = MsmsSpectrum("123",precursor_mz=1020,precursor_charge=2, mz=mz, intensity=intensity_norm)

In [ ]:
import math
def get_num_bins(min_mz: float, max_mz: float, bin_size: float) -> int:
    """
    Compute the number of bins over the given mass range for the given bin
    size.

    Parameters
    ----------
    min_mz : float
        The minimum m/z to include in the vector.
    max_mz : float
        The maximum m/z to include in the vector.
    bin_size : float
        The bin size in m/z used to divide the m/z range.

    Returns
    -------
    int
        The number of bins over the given mass range for the given bin size.
    """
    return math.ceil((max_mz - min_mz) / bin_size)

In [ ]:
print(get_num_bins(50.5,2500,1.0005079))

In [ ]:
import scipy.sparse as ss
import scipy.sparse.linalg

def to_vector(spectrum_mz: np.ndarray, spectrum_intensity: np.ndarray,
              min_mz: float, bin_size: float, num_bins: int)\
        -> ss.csr_matrix:
    """
    Convert the given spectrum to a binned sparse SciPy vector.

    Parameters
    ----------
    spectrum_mz : np.ndarray
        The peak m/z values of the spectrum to be converted to a vector.
    spectrum_intensity : np.ndarray
        The peak intensities of the spectrum to be converted to a vector.
    min_mz : float
        The minimum m/z to include in the vector.
    bin_size : float
        The bin size in m/z used to divide the m/z range.
    num_bins : int
        The number of elements of which the vector consists.

    Returns
    -------
    ss.csr_matrix
        The binned spectrum vector.
    """
    bins = ((spectrum_mz - min_mz) / bin_size).astype(np.int32)
    # bins = np.append(bins,837)
    # spectrum_intensity = np.append(spectrum_intensity,0.5)
    # print(bins)
    print(num_bins)
    vector = ss.csr_matrix(
        (spectrum_intensity, (np.repeat(0, len(spectrum_intensity)), bins)),
        shape=(1, num_bins), dtype=np.float32)
    return vector
    # return vector / scipy.sparse.linalg.norm(vector)

In [ ]:
vector = to_vector(spectrum_mz=spectrum.mz, spectrum_intensity=spectrum.intensity, min_mz=50.5, bin_size=1.0005079, num_bins=2449)

In [ ]:
for a in vector:
    print(a)

In [ ]:
spectrum.mz

In [ ]:
vector.shape

In [ ]:
AA_LIST = [['-', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] ,]
MAX_LEN= 125
# AA_LIST = np.reshape(AA_LIST,(-1,1))
# AA_LIST = list(AA_LIST)
# AA_LIST

In [ ]:
seq = 'MDR'

In [ ]:
seq_pad = seq[:MAX_LEN] if len(seq) > MAX_LEN else seq + '-' * (MAX_LEN - len(seq)) 

In [ ]:
seq_df = pd.DataFrame([{'aa': x} for i, x in enumerate(seq_pad)]) 


In [ ]:
import sklearn.preprocessing
seq_ohe = sklearn.preprocessing.OneHotEncoder(categories=AA_LIST).fit_transform(seq_df)
# .fit_transform(seq_df)

In [ ]:
seq_df

In [ ]:
seq_ohe

In [ ]:
seq_ohe.shape

In [ ]:
def _check_spectrum_valid(spectrum_mz: np.ndarray, min_peaks: int,
                          min_mz_range: float) -> bool:
    """
    Check whether a spectrum is of high enough quality to be used.

    Parameters
    ----------
    spectrum_mz : np.ndarray
        M/z peaks of the sspectrum whose quality is checked.
    min_peaks : int
        Minimum number of peaks a spectrum has to contain.
    min_mz_range : float
        Minimum m/z range the spectrum's peaks need to cover.

    Returns
    -------
    bool
        True if the spectrum has enough peaks covering a wide enough mass
        range, False otherwise.
    """
    return (len(spectrum_mz) >= min_peaks and
            spectrum_mz[-1] - spectrum_mz[0] >= min_mz_range)

In [ ]:
from typing import Optional

def preprocess(spectrum: MsmsSpectrum,
               mz_min: float,
               mz_max: float,
               min_peaks: int,
               min_mz_range: float,
               remove_precursor_tolerance: Optional[float],
               min_intensity: float,
               max_peaks_used: int,
               scaling: Optional[str]) -> MsmsSpectrum:
    """
    Preprocess the given spectrum.

    Parameters
    ----------
    spectrum:  MsmsSpectrum
        The spectrum to be preprocessed.
    mz_min : float
        The minimum m/z value to be included.
    mz_max : float
        The maximum m/z value to be included.
    min_peaks : int
        Minimum number of peaks the spectrum needs to have to be considered
        valid.
    min_mz_range : float
        Minimum m/z range to be covered for the spectrum to be considered
        valid.
    remove_precursor_tolerance : Optional[float]
        Remove peaks within the given m/z of the precursor peak.
    min_intensity : float
        Discard peaks below the given minimum intensity.
    max_peaks_used : int
        Retain only the given number of most intense peaks.
    scaling : Optional[str]
        Perform optional intensity scaling.

    Returns
    -------
    np.ndarray
        The normalized peak intensities.
    """
    if spectrum.is_processed:
        return spectrum

    spectrum = spectrum.set_mz_range(mz_min, mz_max)
    if not _check_spectrum_valid(spectrum.mz, min_peaks, min_mz_range):
        spectrum.is_valid = False
        spectrum.is_processed = True
        return spectrum

    if remove_precursor_tolerance is not None:
        spectrum = spectrum.remove_precursor_peak(
            remove_precursor_tolerance, 'Da')
        if not _check_spectrum_valid(spectrum.mz, min_peaks, min_mz_range):
            spectrum.is_valid = False
            spectrum.is_processed = True
            return spectrum

    spectrum = spectrum.filter_intensity(min_intensity, max_peaks_used)
    if not _check_spectrum_valid(spectrum.mz, min_peaks, min_mz_range):
        spectrum.is_valid = False
        spectrum.is_processed = True
        return spectrum

    scaling = scaling
    if scaling == 'sqrt':
        scaling = 'root'
    if scaling is not None:
        spectrum = spectrum.scale_intensity(scaling, max_rank=max_peaks_used)
    print(spectrum.intensity)
    # spectrum.intensity = _norm_intensity(spectrum.intensity)
    inten = _norm_intensity(spectrum.intensity)
    # Set a flag to indicate that the spectrum has been processed to avoid
    # reprocessing.
    spectrum.intensity = inten
    spectrum.is_valid = True
    spectrum.is_processed = True

    return spectrum

In [ ]:
spectrum = preprocess(spectrum=spectrum, mz_min=50.5, mz_max=2500, min_mz_range=10,min_peaks=20, remove_precursor_tolerance=0.5, min_intensity=0.1, max_peaks_used=50, scaling='sqrt')

In [ ]:
spectrum.mz

In [ ]:
spectrum._intensity = [1,2,3,4]

In [ ]:
vector1 = to_vector(spectrum.mz, spectrum.intensity, min_mz=50.5, bin_size=1.0005079, num_bins=2449)

In [ ]:
for a in vector1:
    print(a)

In [278]:
import numpy as np
import math
from tensorflow.keras.utils import Sequence

class Dataloader(Sequence):

    def __init__(self, x_set, y_set, batch_size, shuffle=False):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.shuffle=shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

		# batch 단위로 직접 묶어줘야 함
    def __getitem__(self, idx):
				# sampler의 역할(index를 batch_size만큼 sampling해줌)
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        batch_x = [self.x[i] for i in indices]
        batch_y = [self.y[i] for i in indices]

        return np.array(batch_x), np.array(batch_y)

    # epoch이 끝날때마다 실행
    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indices)
            

# train_loader = Dataloader(x, y, 128, shuffle=True)
# valid_loader = Dataloader(x, y, 128)
# test_loader = Dataloader(x, y, 128)

# 방법 1
# model.fit(train_loader, validation_data=valid_loader, epochs=10,
# 					workers=4)# multi로 처리할 개수
# model.evaluate(test_loader)


In [279]:
Sequence

keras.src.utils.data_utils.Sequence